In [16]:
import requests
import pandas as pd
import time
import json
import shelve
from math import degrees, radians, sin, cos, sqrt, atan2

#Maps API key
API_KEY = ''

# Starting coordinates for LES pre-transform
top_left = (40.724353, -73.992521)
top_right = (40.724353, -73.980233)
bottom_right = (40.716773, -73.980233)
bottom_left = (40.716773, -73.992521)

# Radius of the circles in meters
circle_radius_meters = 35.405
circle_radius_meters_api = 43

# Rotation angle in radians
rotation_angle = radians(17)

def meters_to_degrees(meters):
    # Convert meters to degrees using haversine formula
    earth_radius_meters = 6371000  # Earth's radius in meters
    distance_in_radians = meters / earth_radius_meters
    distance_in_degrees = degrees(distance_in_radians)
    return distance_in_degrees

def fetch_places(lat, lng, radius, page_token=None):
    cache_key = f'{lat},{lng},{radius},{page_token}'
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&key={}'.format(lat, lng, radius, API_KEY)
    if page_token:
        url += '&pagetoken={}'.format(page_token)
    #print(url)
        
    with shelve.open('places_cache.db') as cache:  
        if cache_key in cache:
            res_json = cache[cache_key]
        else:
            response = requests.get(url)
            res_json = response.json()
            cache[cache_key] = res_json
            time.sleep(2.5)  # Sleep for 3 seconds to respect rate limit
            
        with open('api_calls.json', 'a') as f:
            json.dump({
                'url': url,
                'response': res_json
            }, f)
            f.write('\n')  # Add newline for next entry
        
        if 'results' in res_json and res_json['results']:
            return res_json
        else:
            print("API Error")
            print(res_json)
            print("\n\n\n")
            return None

def populate_places(lat, lng, radius):
    if radius < 10:
        # Minimum radius reached, print message and return
        print("Minimum radius reached. Lat: {}, Lng: {}, Radius: {}".format(lat, lng, radius))
        return
    print("sanity")
    page_token = None
    page_count = 1
    res_json = fetch_places(lat, lng, radius, page_token)
    while res_json is not None:
        for place in res_json['results']:
            yield place
        page_token = res_json.get('next_page_token')
        if page_token:
            res_json = fetch_places(lat, lng, radius, page_token)
            page_count += 1
            
            if page_count == 3 and len(res_json["results"]) == 20:
                print("60 listings, may need to subdivide")
                # Subdivide the circle into 8 smaller circles
                subdivisions = 8
                new_radius = radius / 2
                
                for i in range(subdivisions):
                    angle = i * 45  # 45 degrees between subdivisions
                    lat_sector = lat + (meters_to_degrees(new_radius) * cos(angle))
                    lng_sector = lng + (meters_to_degrees(new_radius) * sin(angle))
                    print("Subdividing into {}, {} - {}", lat_sector, lng_sector, new_radius)
                    yield from populate_places(lat_sector, lng_sector, new_radius)
            else:
                # Continue with the next page
                continue
        else:
            # No more pages, return
            res_json = None
            return


def haversine_distance(coord1, coord2):
    # Haversine distance calculation
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon/2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    R = 6371000  # Earth's radius in meters
    distance = R * c

    return distance

def rotate_point(point, center, angle):
    # Rotation of the point around the center by a given angle
    x, y = point
    cx, cy = center
    rotated_x = (x - cx) * cos(angle) - (y - cy) * sin(angle) + cx
    rotated_y = (x - cx) * sin(angle) + (y - cy) * cos(angle) + cy
    return rotated_x, rotated_y

def generate_circle_centers(top_left, top_right, bottom_right, bottom_left, circle_radius_meters, rotation_angle):
    # Code to generate circle centers goes here
    width = haversine_distance(top_left, top_right)
    height = haversine_distance(top_left, bottom_left)

    num_circles_x = int(width / (2 * circle_radius_meters))
    num_circles_y = int(height / (2 * circle_radius_meters))

    circle_locations = []
    for i in range(num_circles_x + 1):
        for j in range(num_circles_y + 1):
            # Calculate the latitude offset in degrees for each circle
            lat_offset_meters = j * (height / num_circles_y)
            lat_offset_degrees = lat_offset_meters / 111111

            # Calculate the new latitude for the circle center
            circle_center_lat = top_left[0] - lat_offset_degrees

            # Calculate the longitude offset in degrees for each circle
            lon_offset_meters = i * (width / num_circles_x)
            lon_offset_degrees = lon_offset_meters / (111111 * cos(radians(circle_center_lat)))

            # Calculate the new longitude for the circle center
            circle_center_lon = top_left[1] + lon_offset_degrees

            circle_center = (circle_center_lat, circle_center_lon)

            # Rotate the circle center around the top left corner
            rotated_circle_center = rotate_point(circle_center, top_left, rotation_angle)

            circle_locations.append(rotated_circle_center)

    return circle_locations

def get_coordinates():
    return [(40.7189838, -73.9893590)]

def get_place_details(place_id):
    api_key = API_KEY
    url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}'

    response = requests.get(url)
    if response.status_code == 200:
        place_details = response.json()
        with open('places_calls.json', 'a') as f:
            json.dump({
                'url': url,
                'response': place_details
            }, f)
            f.write('\n')  # Add newline for next entry
    else:
        print(f"Error: Unable to fetch place details for place ID {place_id}")


# coordinates = get_coordinates()
coordinates = generate_circle_centers(top_left, top_right, bottom_right, bottom_left, circle_radius_meters, rotation_angle)

places = []

for lat, lng in coordinates:
    places.extend(list(populate_places(lat, lng, circle_radius_meters_api)))

# Convert the list of places to a DataFrame
df = pd.DataFrame(places)

# Remove duplicates
df = df.drop_duplicates(subset=['name', 'vicinity'])

df['google_maps_url'] = 'https://www.google.com/maps/place/?q=place_id:' + df['place_id']

# Remove permanently closed
df = df[df['business_status'] == 'OPERATIONAL']

# Delete the specified columns
columns_to_delete = ['geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'reference', 'scope', 'opening_hours', 'plus_code']
df.drop(columns=columns_to_delete, inplace=True)

df.to_csv('places-raw.csv', index=False)
# Filters
# Remove those that are only listed as 'establishment' and nothing else
df = df[~(df['types'].apply(lambda x: len(x) == 1 and 'establishment' in x))]

# Remove rows containing 'route' in the 'types' column
df = df.loc[~df['types'].apply(lambda x: 'route' in x)]

# Remove rows containing 'locality' in the 'types' column
df = df.loc[~df['types'].apply(lambda x: 'locality' in x)]

# Remove rows containing 'transit_station' in the 'types' column
df = df.loc[~df['types'].apply(lambda x: 'transit_station' in x)]

#lawyer, church, real_estate_agency, doctor, beauty_salon, travel, finance, parking, insurance_agency, bank, general_contractor, lodging
df = df.loc[~df['types'].apply(lambda x: 'lawyer' in x)]
df = df.loc[~df['types'].apply(lambda x: 'church' in x)]
df = df.loc[~df['types'].apply(lambda x: 'real_estate_agency' in x)]
df = df.loc[~df['types'].apply(lambda x: 'doctor' in x)]
df = df.loc[~df['types'].apply(lambda x: 'beauty_salon' in x)]
df = df.loc[~df['types'].apply(lambda x: 'finance' in x)]
df = df.loc[~df['types'].apply(lambda x: 'parking' in x)]
df = df.loc[~df['types'].apply(lambda x: 'insurance_agency' in x)]
df = df.loc[~df['types'].apply(lambda x: 'bank' in x)]
df = df.loc[~df['types'].apply(lambda x: 'general_contractor' in x)]
df = df.loc[~df['types'].apply(lambda x: 'lodging' in x)]

df = df.loc[~((df['types'].apply(lambda x: ['point_of_interest', 'establishment'] == x)) & (df['types'].apply(len) == 2))]
df = df.loc[~((df['types'].apply(lambda x: ['establishment', 'point_of_interest'] == x)) & (df['types'].apply(len) == 2))]

# Save the DataFrame to a CSV file
df.to_csv('places-cleaned.csv', index=False)

# get_place_details('ChIJAQAQdIdZwokRM_yzRfJphns')

sanity
sanity
sanity
sanity
sanity
sanity
sanity
sanity
sanity
sanity
60 listings, may need to subdivide
Subdividing into {}, {} - {} 40.71861104612729 -73.994335605765 21.5
sanity
Subdividing into {}, {} - {} 40.71851926516616 -73.99417108004131 21.5
sanity
Subdividing into {}, {} - {} 40.71833105509095 -73.99416274780424 21.5
sanity
Subdividing into {}, {} - {} 40.71822509427004 -73.99431851931323 21.5
sanity
Subdividing into {}, {} - {} 40.71830197724689 -73.99449051194811 21.5
sanity
Subdividing into {}, {} - {} 40.71848871470441 -73.99451544346516 21.5
sanity
Subdividing into {}, {} - {} 40.71860802631271 -73.9943696449785 21.5
sanity
Subdividing into {}, {} - {} 40.718546642877904 -73.99419153115952 21.5
sanity
sanity
sanity
sanity
sanity
sanity
sanity
60 listings, may need to subdivide
Subdividing into {}, {} - {} 40.72231111346538 -73.99228589834827 21.5
sanity
Subdividing into {}, {} - {} 40.722219332504245 -73.99212137262458 21.5
sanity
Subdividing into {}, {} - {} 40.7220311